Data Preparation

In [1]:
!pwd
!ls

/content
drive  sample_data


In [ ]:
path = "/content/drive/MyDrive/ELEC301/data"
os.chdir(path)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [ ]:
# Time Domain Data
data_t = []
label = []
for filename in os.listdir():
    fs, data = wavfile.read(filename)
    if len(data.shape) > 1:
        data = data[:, 0]
    data = data.astype(float)
    data_t.append(data)
    label.append(str(filename[: -7]))

In [ ]:
threshold = 0.1
cut = []
for i in range(1125):
  effect = data_t[i][np.argwhere(data_t[i]/max(data_t[i]) > threshold)[0,0]:np.argwhere(data_t[i]/max(data_t[i]) > threshold)[-1,0]+1]
  cut.append(effect)

In [ ]:
from IPython import display
display.display(display.Audio(data_pad[24,0], rate=48000))
display.display(display.Audio(D[400,1], rate=48000))
display.display(display.Audio(D[50,2], rate=48000))
display.display(display.Audio(D[2,3], rate=48000))

In [ ]:
# Put Data Into 2D Matrix
D = []
maxl = 0
for i in range(1125):
  print(len(cut[i]))
  seg_data = []
  for j in range(6):
    data = cut[i]
    seg = int(len(data)/(6))
    if j < (5):
      seg_data.append(data[j*seg:j*seg+seg])
    else:
      seg_data.append(data[5*seg:-1])
  D.append(seg_data)
D = np.array(D)

In [ ]:
for i in range(1125):
  for j in range(6):
    if len(D[i,j]) > maxl:
      maxl = len(D[i,j])
maxl

In [ ]:
data_pad = [] # Time Domain Data
for i in range(1125):
  zero_pad = np.zeros((6, maxl))
  for j in range(6):
    zero_pad[j, range(0, len(D[i,j]))] = D[i,j]
  data_pad.append(zero_pad)

In [ ]:
data_pad = np.array(data_pad)
data_pad.shape

In [ ]:
# Make Number Label (1125)
Y = []
for emo in range(0, len(label)):
    if label[emo] == 'angry':
        Y.append(0)
    if label[emo] == 'calm':
        Y.append(1)
    if label[emo] == 'disgust':
        Y.append(2)
    if label[emo] == 'fearful':
        Y.append(3)
    if label[emo] == 'happy':
        Y.append(4)
    if label[emo] == 'neutral':
        Y.append(5)
    if label[emo] == 'sad':
        Y.append(6)
    if label[emo] == 'surprised':
        Y.append(7)
Y = np.array(Y)

Spectral Process

In [ ]:
import librosa
import librosa.display

In [ ]:
# MFCC
MFCC = np.zeros((1125,54,54,6))
for i in range(1125):
  print(i)
  for j in range(6):
    MFCC[i,:,:,j] = (librosa.feature.mfcc(y=data_pad[i,j], sr=fs, n_mfcc = 54))

In [ ]:
# STFT
STFT = np.zeros((1125,65,80,4))
for i in range(1125):
  for j in range(4):
    STFT[i,:,:,j] = np.abs(librosa.stft(data_pad[i,j],n_fft=128,hop_length=512))

In [ ]:
# Chroma
Chroma = np.zeros((1125,54,54,6))
for i in range(1125):
  for j in range(6):
    Chroma[i,:,:,j] = (librosa.feature.chroma_stft(y=data_pad[i,j], sr=fs, n_chroma = 54))

/usr/local/lib/python3.8/dist-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


In [ ]:
# MelSpectrogram
MELS = np.zeros((1125,54,54,6))
for i in range(1125):
  for j in range(4):
    MELS[i,:,:,j] = (librosa.feature.melspectrogram(y=data_pad[i,j], sr=fs, n_mels = 54))

In [ ]:
print(MELS.shape)
print(Chroma.shape)
print(MFCC.shape)


In [ ]:
DATA = np.zeros((1125, 54, 54, 12))
for i in range(1125):
  for j in range(6):
    #DATA[i*4+j,:,:,0] = MFCC[i,:,:,j]
    DATA[i,:,:,j] = MFCC[i,:,:,j]
    DATA[i,:,:,j+6] = Chroma[i,:,:,j]

In [ ]:
DATA.shape

In [ ]:
# Make Number Label
Y = []
for emo in range(0, len(label)):
    if label[emo] == 'angry':
      for i in range(4):
        Y.append(0)
    if label[emo] == 'calm':
      for i in range(4):
        Y.append(1)
    if label[emo] == 'disgust':
      for i in range(4):
        Y.append(2)
    if label[emo] == 'fearful':
      for i in range(4):
        Y.append(3)
    if label[emo] == 'happy':
      for i in range(4):
        Y.append(4)
    if label[emo] == 'neutral':
      for i in range(4):
        Y.append(5)
    if label[emo] == 'sad':
      for i in range(4):
        Y.append(6)
    if label[emo] == 'surprised':
      for i in range(4):
        Y.append(7)
Y = np.array(Y)

Train/Test Set Making

In [ ]:
# Select Training and Test Data
random_traning_data = np.random.choice(len(Y), 900, replace=False)
train_data = DATA[random_traning_data]
train_label = Y[random_traning_data]
remaining_data = np.setdiff1d(range(0, len(Y)), random_traning_data)
test_data = DATA[remaining_data]
test_label = Y[remaining_data]
len(test_label)+len(train_label)

CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
input_shape = DATA.shape[1:]
num_labels = 8
# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
norm_layer.adapt(train_data)

model = models.Sequential([
    layers.Input(shape=input_shape),

    #Downsample the input.
    #layers.Resizing(64, 64),

    norm_layer,
    layers.Conv2D(32, 2, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 2, activation='relu'),
    layers.MaxPooling2D(),
    #layers.Conv2D(64, 2, activation='relu'),
    #layers.MaxPooling2D(),
    #layers.Conv2D(32, 2, activation='relu'),
    #layers.MaxPooling2D(),
    #layers.Conv2D(32, 2, activation='relu'),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])
model.summary()

In [ ]:

  random_traning_data = np.random.choice(len(Y), 900, replace=False)
  train_data = DATA[random_traning_data]
  train_label = Y[random_traning_data]
  remaining_data = np.setdiff1d(range(0, len(Y)), random_traning_data)
  test_data = DATA[remaining_data]
  test_label = Y[remaining_data]
  len(test_label)+len(train_label)
  model.compile(
      optimizer=tf.keras.optimizers.Adam(),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])
#history = model.fit(train_data, train_label, epochs=10, validation_data=(test_data, test_label), callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2))
  history = model.fit(train_data, train_label, epochs=5, validation_data=(test_data, test_label))

Prediction

In [ ]:
path = "/content/drive/MyDrive/ELEC301/test"
os.chdir(path)

In [ ]:
!unzip 'test.zip'

In [ ]:
# Time Domain Data
data_test = []
label_test = []
NUM = []
L_test = [] # length of each audio
for filename in os.listdir():
    label_test.append(str(filename[: -7]))
    fs, data = wavfile.read(filename)
    if len(data.shape) > 1:
        data = data[:, 0]
    data = data.astype(float)
    data_test.append(data)
    L_test.append(len(data))
    NUM.append(str(filename[6: 9]))
print(L_test)

In [ ]:
for i in range(315):
  data_test[i] = data_test[i].astype(float)

In [ ]:
cut = []
for i in range(315):
  posi = np.argmax(data_test[i])
  small = np.argwhere(data_test[i]/data_test[i][posi] > 0.1)[0,0]
  big = np.argwhere(data_test[i]/data_test[i][posi] > 0.1)[-1,0]
  eff = data_test[i][small : big+1]
  cut.append(eff)

In [ ]:
# Put Data Into 2D Matrix
D_test = []
for i in range(315):
  seg_data = []
  for j in range(6):
    data = cut[i]
    seg = int(len(data)/(6))
    if j < (5):
      seg_data.append(data[j*seg:j*seg+seg])
    else:
      seg_data.append(data[5*seg:-1])
  D_test.append(seg_data)

In [ ]:
D_test = np.array(D_test)

In [ ]:
data_pad1 = [] # Time Domain Data
for i in range(315):
  zero_pad = np.zeros((6, maxl))
  for j in range(6):
    zero_pad[j, range(0, len(D_test[i,j]))] = D_test[i,j]
  data_pad1.append(zero_pad)

In [ ]:
data_pad1 = np.array(data_pad1)
data_pad1.shape

In [ ]:
# MFCC
MFCCt = np.zeros((315,54,54,6))
Chromat = np.zeros((315,54,54,6))
MELSt = np.zeros((315,54,54,6))
for i in range(315):
  for j in range(6):
    MFCCt[i,:,:,j] = (librosa.feature.mfcc(y=data_pad1[i,j], sr=fs,n_mfcc=54))
    Chromat[i,:,:,j] = (librosa.feature.chroma_stft(y=data_pad1[i,j], sr=fs, n_chroma = 54))
    MELSt[i,:,:,j] = (librosa.feature.melspectrogram(y=data_pad1[i,j], sr=fs, n_mels = 54))


In [ ]:
DATAt = np.zeros((315, 54, 54, 12))
for i in range(315):
  for j in range(6):
    #DATA[i*4+j,:,:,0] = MFCC[i,:,:,j]
    DATAt[i,:,:,j] = MFCCt[i,:,:,j]
    DATAt[i,:,:,j+6] = Chromat[i,:,:,j]

In [ ]:
predictions = model.predict(DATAt)
score = tf.nn.softmax(predictions)
label_predict = []
for i in range(315):
  label_predict.append(np.argmax(score[i]))
res = []
for emo in range(0, len(label_predict)):
    if label_predict[emo] == 0:
      res.append('angry')
    if label_predict[emo] == 1:
      res.append('calm')
    if label_predict[emo] == 2:
      res.append('disgust')
    if label_predict[emo] == 3:
      res.append('fearful')
    if label_predict[emo] == 4:
      res.append('happy')
    if label_predict[emo] == 5:
      res.append('neutral')
    if label_predict[emo] == 6:
      res.append('sad')
    if label_predict[emo] == 7:
      res.append('surprised')
print(res)
res = np.array(res).transpose()
NUM = np.array(NUM).transpose()
sample = np.column_stack([NUM, res])
sort_sample = sample[sample[:, 0].argsort()]
name = []
for item in sort_sample[:,0]:
  item = 'sample'+str(item)
  name.append(item)
name = np.array(name).transpose()
sort_sample[:,0] = name
sort_sample

In [ ]:
res_chroma = sort_sample

In [ ]:
import pandas as pd
DF = pd.DataFrame(sort_sample)
DF.to_csv("res1201_all.csv")